In [18]:
# XGBoost: XGBoost is an implementation of gradient boosted decision trees designed for speed and performance.
# Gradient Boosting algorithm also called gradient boosting machine including the learning rate.
# Stochastic Gradient Boosting with sub-sampling at the row, column and column per split levels.
# Regularized Gradient Boosting with both L1 and L2 regularization.

In [19]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

%matplotlib inline

In [20]:
# Importing the dataset
df = pd.read_csv("../../archive/Churn_Modelling.csv")

In [21]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [22]:
# Making Y-Variable
y = df['Exited']

# Including all but one column
Xs = df.iloc[:, 3:13]

In [23]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

labelencoder_X_geo = LabelEncoder()
Xs.loc[:, "Geography"] = labelencoder_X_geo.fit_transform(Xs.loc[:, "Geography"])
labelencoder_X_gender = LabelEncoder()
Xs.loc[:, "Gender"] = labelencoder_X_gender.fit_transform(Xs.loc[:, "Gender"])

onehotencoder = OneHotEncoder(categorical_features=[Xs.columns.get_loc("Geography")])
Xs = onehotencoder.fit_transform(Xs).toarray()

In [24]:
Xs[0]

array([1.0000000e+00, 0.0000000e+00, 0.0000000e+00, 6.1900000e+02,
       0.0000000e+00, 4.2000000e+01, 2.0000000e+00, 0.0000000e+00,
       1.0000000e+00, 1.0000000e+00, 1.0000000e+00, 1.0134888e+05])

In [25]:
# Removing one of the dummy variables for the country
Xs = Xs[:, 1:]

In [26]:
# Splitting the data set into training and testing

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(Xs, y, test_size=0.2, random_state=0)

In [27]:
# Importing the XGboost
from xgboost import XGBClassifier
classifier = XGBClassifier()
classifier.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [28]:
# Predicting the training set
y_pred = classifier.predict(X_test)

In [29]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = cm.ravel()

total = tn+fp+fn+tp
accuracy = (tn+tp)/total
precision = tp/(tp+fp)
recall = tp/(tp+fn)
f1_score = 2*precision*recall/(precision+recall)


print(
    "true pos: {0}\n"
    "false pos: {1}\n"
    "true neg: {2}\n"
    "false neg: {3}\n".format(tp, fp, tn, fn))

print("""
Out of {0} reviews, the model got {1} correct,
Accuacy is: {2:.2f}%
Precision is: {3:.2f}%
Recall is: {4:.2f}%
F1 Score is: {5:.2f}%""".format(total, tn+tp, accuracy, precision, recall, f1_score))

true pos: 208
false pos: 74
true neg: 1521
false neg: 197


Out of 2000 reviews, the model got 1729 correct,
Accuacy is: 0.86%
Precision is: 0.74%
Recall is: 0.51%
F1 Score is: 0.61%


In [30]:
X_train

array([[0.0000000e+00, 1.0000000e+00, 6.6700000e+02, ..., 1.0000000e+00,
        0.0000000e+00, 1.6383064e+05],
       [1.0000000e+00, 0.0000000e+00, 4.2700000e+02, ..., 1.0000000e+00,
        1.0000000e+00, 5.7098000e+04],
       [0.0000000e+00, 0.0000000e+00, 5.3500000e+02, ..., 1.0000000e+00,
        0.0000000e+00, 1.8563076e+05],
       ...,
       [0.0000000e+00, 0.0000000e+00, 7.3800000e+02, ..., 1.0000000e+00,
        0.0000000e+00, 1.8142987e+05],
       [0.0000000e+00, 1.0000000e+00, 5.9000000e+02, ..., 1.0000000e+00,
        1.0000000e+00, 1.4875016e+05],
       [1.0000000e+00, 0.0000000e+00, 6.2300000e+02, ..., 1.0000000e+00,
        0.0000000e+00, 1.1885526e+05]])

In [31]:
import warnings

In [33]:
from sklearn.model_selection import cross_val_score
warnings.filterwarnings(action='ignore', category=DeprecationWarning)
accuracies = cross_val_score(estimator=classifier, X=X_train, y=y_train, cv=10)
print('Mean: {0} \n Std: {1}'.format(accuracies.mean(),accuracies.std()))

Mean: 0.8629994451163204 
 Std: 0.010677872171663988


In [ ]:
params = {
    'min_child_weight': [1, 5, 10],
    'gamma': [0.5, 1, 1.5, 2, 5],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'max_depth': [3, 4, 5]
}


In [ ]:
# Appyling grid search to find the best models and parameters
from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(estimator=classifier, 
                           param_grid=params,
                           scoring='accuracy',
                           cv=10,
                           n_jobs=-1
                          )

grid_search = grid_search.fit(X_train, y_train)